### Web Scraping Crude Oil News and using VADER for sentiment analysis

In [ ]:
# import necessary libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 23.0 MB/s 


In [ ]:
# Create lists to store scraped news urls, headlines and text
url_list = []
date_time = []
news_text = []
headlines = []

for i in range(1,3): # parameters of range function correspond to page numbers in the website with news listings
    # get the list of unique urls in the page
    url = 'https://oilprice.com/Energy/Crude-Oil/Page-{}.html'.format(i)
    request = requests.get(url)
    soup = BeautifulSoup(request.text, "html.parser")
    for links in soup.find_all('div', {'class': 'categoryArticle'}):
        for info in links.find_all('a'):
            if info.get('href') not in url_list:
                url_list.append(info.get('href'))

for www in url_list:
    # access each url
    headlines.append(www.split("/")[-1].replace('-',' ').replace('.html', ' '))
    request = requests.get(www)
    soup = BeautifulSoup(request.text, "html.parser")

    # store date and time of the event of article publication
    for dates in soup.find_all('span', {'class': 'article_byline'}):
        date_time.append(dates.text.split('-')[-1])
    
    # store the text of the news
    temp = []
    for news in soup.find_all('p'):
        temp.append(news.text)
    """Code which I could not find any reason to include here, but it is included in the reference material:

    #identify the last line of the news article
    # for last_sentence in reversed(temp):
    #     if last_sentence.split(" ")[0]=="By" and last_sentence.split(" ")[-1]=="Oilprice.com":
    #         break
    #     elif last_sentence.split(" ")[0]=="By":
    #         break """
    
    #prune non news related text from the scraped data to create the news text
    joined_text = ' '.join(temp[temp.index("More Info")+1:temp.index(temp[-1])])
    news_text.append(joined_text)

In [ ]:
headlines

['US Gulf Of Mexico Can Help Fill Global Oil Supply Gap ',
 'Oil Jumps On EIA Inventory Data ',
 'OPEC Crude Oil Exports Trend Lower In September ',
 'Oil Price Cap On Russian Crude Could Cause Tanker Shortage ',
 'Russia Likely To Propose Major Output Cut At Next OPEC Meeting ',
 'Russia At The Forefront Of Development Of Huge Iranian Oil Fields ',
 'Not Everyone Is Ditching Russian Energy ',
 'Colombia May Shoot Itself In The Foot With Ban On Fracking ',
 'Oil Rig Count Sees Small Jump Amid Crash In Crude Prices ',
 'An Oil Supply Shock May Be Imminent ',
 'Oil Falls Below 80 As Powell Warns A Recession May Be Looming ',
 'Razor Thin Spare Capacity To Keep Oil Prices Elevated ',
 'How Big A Problem Is Americas Shrinking Oil Reserve ',
 'Why India Is Suddenly Buying Less Russian Crude ',
 'Oil Prices Fall After Fed Raises Rates ',
 'Oil Prices Dip On Across The Board Inventory Build ',
 'Germany Seizes Rosneft Refineries But Doesnt Solve Its Oil Problem ',
 'Oil Under Pressure As Trad

In [ ]:
news_text[0:2]

['One of the world’s most mature deepwater basins, the U.S. Gulf of Mexico, has a lot of production life left and could help bridge the oil supply gap that is widening with the maturing of many currently producing assets globally. The basin now pumping 15% of U.S. crude has the potential to deliver another up to 2 million barrels per day (bpd) of the new supply sources needed by 2040, McKinsey & Company said in a new report this month. The Gulf can also do it with one of the lowest emissions per barrel of all major basins in the world.\xa0\xa0 The deepwater U.S. Gulf of Mexico has one of the lowest emissions-per-barrel estimates in the world, right behind Saudi Arabia and the United Arab Emirates (UAE) and the North Sea deepwater basin, including offshore Norway and the UK, according to McKinsey estimates based on its calculations and data from the International Energy Agency (IEA) and Stanford University.\xa0 Operators in the Gulf of Mexico—most of them supermajors with net-zero 2050 

In [ ]:
# save news text along with the news headline in a dataframe
news_df = pd.DataFrame({
    'Date': date_time,
    'Headline': headlines,
    'News': news_text
})
news_df.head()

,Date,Headline,News
0,"Sep 28, 2022, 5:00 PM CDT",US Gulf Of Mexico Can Help Fill Global Oil Sup...,One of the world’s most mature deepwater basin...
1,"Sep 28, 2022, 9:38 AM CDT",Oil Jumps On EIA Inventory Data,Crude oil prices moved higher today after the ...
2,"Sep 28, 2022, 9:00 AM CDT",OPEC Crude Oil Exports Trend Lower In September,Crude oil exports from all OPEC oil producers ...
3,"Sep 27, 2022, 4:00 PM CDT",Oil Price Cap On Russian Crude Could Cause Tan...,With the EU embargo on imports of Russian oil ...
4,"Sep 27, 2022, 9:48 AM CDT",Russia Likely To Propose Major Output Cut At N...,Russia is likely to propose at the next OPEC+ ...


In [ ]:
len(news_df)

40

In [ ]:
# use VADER to perform sentiment analysis on stored news articles
analyser = SentimentIntensityAnalyzer()

def comp_score(text):
    return analyser.polarity_scores(text)['compound']

news_df['sentiment'] = news_df['News'].apply(comp_score)

news_df.head()

,Date,Headline,News,sentiment
0,"Sep 28, 2022, 5:00 PM CDT",US Gulf Of Mexico Can Help Fill Global Oil Sup...,One of the world’s most mature deepwater basin...,0.8070
1,"Sep 28, 2022, 9:38 AM CDT",Oil Jumps On EIA Inventory Data,Crude oil prices moved higher today after the ...,-0.9033
2,"Sep 28, 2022, 9:00 AM CDT",OPEC Crude Oil Exports Trend Lower In September,Crude oil exports from all OPEC oil producers ...,-0.9838
3,"Sep 27, 2022, 4:00 PM CDT",Oil Price Cap On Russian Crude Could Cause Tan...,With the EU embargo on imports of Russian oil ...,-0.9144
4,"Sep 27, 2022, 9:48 AM CDT",Russia Likely To Propose Major Output Cut At N...,Russia is likely to propose at the next OPEC+ ...,-0.9619


In [ ]:
# According to vader documentation, sentiment between -0.05 to 0.05 considered neutral, from 0.05 onwards considered positive and less than
# 0.05 considered negative
news_df['Sentiment'] = news_df['sentiment'].apply(lambda x: 'positive' if x >=0.05 else 'negative' if x <= -0.05 else 'neutral')

In [ ]:
news_df.Sentiment.value_counts()

negative    33
positive     7
Name: Sentiment, dtype: int64

In [ ]:
from google.colab import files
news_df.to_csv("crudeoil_news_articles.csv", index=False)
files.download('crudeoil_news_articles.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>